In [1]:
import pandas as pd
import json
import requests

C:\Users\junke\AppData\Local\Temp\ipykernel_19392\705113211.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# URL of the JSON data
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"

# Fetch data from the URL
response = requests.get(url)

# Parse JSON data
restaurant_data = response.json()

# Normalize "restaurants" object in JSON data to DataFrame    
restaurant_df = pd.json_normalize(restaurant_data, 'restaurants')

# Read in Country-Code Excel file into Dataframe
country_code_df = pd.read_csv("Country-Code.csv")

In [3]:
# pd.set_option('display.max_columns', None)

In [4]:
# Merge restaurant data and country code excel file based on country id
merged_restaurant_df = pd.merge(restaurant_df, country_code_df, 
                                how = 'left', left_on = 'restaurant.location.country_id', 
                                right_on='Country Code')

# Extract required fields
final_restaurant_df = merged_restaurant_df[['restaurant.R.res_id', 'restaurant.name',
                                             'Country', 'restaurant.location.city',
                                             'restaurant.user_rating.votes', 'restaurant.user_rating.aggregate_rating',
                                             'restaurant.cuisines']]

# Rename columns
renamed_columns = {'restaurant.R.res_id':'Restuarant Id', 'restaurant.name':'Restaurant Name', 
                   'restaurant.location.city':'City', 'restaurant.user_rating.votes':'User Rating Votes', 
                   'restaurant.user_rating.aggregate_rating':'User Aggregate Rating', 'restaurant.cuisines':'Cuisines' }
final_restaurant_df = final_restaurant_df.rename(columns=renamed_columns)

#Convert User Aggregate Rating Column to a float data type
final_restaurant_df['User Aggregate Rating'] = final_restaurant_df['User Aggregate Rating'].astype('float64') 

In [11]:
def export_to_csv(file_path):
    """
    Export DataFrame to a CSV file and append it to the existing file.
    
    Parameters:
        df: The DataFrame to export.
        file_path: The path to the CSV file where the DataFrame will be saved.
    """
    # Export DataFrame to CSV and append it to the specified file
    final_restaurant_df.to_csv(file_path, index=False)  

# Input file path to export restaurant.csv file
export_to_csv("C:\\Users\\junke\\Desktop\\Important Documents (JunKeat)\\GovTech-Application\\restaurants.csv")